In [4]:
from geoguessr import Geoguessr
from geoguessr.util import clean, refresh_dotenv
from dotenv import load_dotenv
import os

load_dotenv(r"/Users/davidbirkenberger/Projects/geoliga/tests/.venv")
cookie = os.getenv("GEOGUESSR_COOKIE")
if not cookie:
    raise RuntimeError("GEOGUESSR_COOKIE fehlt. Bitte setze die Umgebungsvariable.")

# Extract the actual cookie value (remove _ncfa= prefix if present)
if cookie.startswith("_ncfa="):
    cookie = cookie[6:]  # Remove "_ncfa=" prefix
    
# initialize our instance of the class with the cookie
geo = Geoguessr(ncfa_cookie=cookie)

In [5]:
# make the html request to API
challenge_id = "yeNlf2aGuU1sqY7L"

# Let's first test if we can make a simple request to debug
import requests

# Test the API endpoint directly to see what we get
url = f"https://www.geoguessr.com/api/v3/results/scores/{challenge_id}/0/26"
print(f"Testing URL: {url}")

# Get headers from the geoguessr instance
headers = geo._get_headers()
print(f"Headers: {headers}")

# Make a test request
response = requests.get(url, headers=headers)
print(f"Status code: {response.status_code}")
print(f"Response text (first 200 chars): {response.text[:200]}")
print(f"Content-Type: {response.headers.get('content-type', 'Not set')}")

# Now try the library method
try:
    raw_data = geo.get_challenge_scores(challenge_id)
    print(f"Success! Got data: {type(raw_data)}")
    scores = clean(raw_data)
    print(f"Cleaned scores: {scores}")
except Exception as e:
    print(f"Error with library method: {e}")
    print(f"Error type: {type(e)}")

Testing URL: https://www.geoguessr.com/api/v3/results/scores/yeNlf2aGuU1sqY7L/0/26
Headers: {'authority': 'www.geoguessr.com', 'accept': '*/*', 'accept-language': 'en-US,en;q=0.9', 'content-type': 'application/json', 'cookie': '{YgEDeFLDtus9Sl1uPM%2F5uh1tihZuF6Wh4a%2BVjBmvCJk%3DbH13%2FmABJMpsvyB05GPf0zN3ocCWQwc6Y1apZo8QzRydns%2FPA82izn%2FIwBoj6mAjZxGyuM6m0yNgInTcsR2AFplw2rH%2FYPHoRg7gLf03ce8%3D}', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.99 Safari/537.36'}
Status code: 404
Response text (first 200 chars): 
Content-Type: Not set
Error with library method: Expecting value: line 1 column 1 (char 0)
Error type: <class 'requests.exceptions.JSONDecodeError'>


In [12]:
# Let's try the alternative endpoints from your hello.py file
challenge_id = "FbxiQzxzq9XuwwY2"

# Test different API endpoints that might work
endpoints_to_try = [
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/leaderboard", 
    f"https://www.geoguessr.com/api/v3/results/{challenge_id}",
    f"https://www.geoguessr.com/api/v3/results/highscores/{challenge_id}",
    f"https://www.geoguessr.com/api/v3/results/scores/{challenge_id}/0/26"
]

headers = geo._get_headers()

for endpoint in endpoints_to_try:
    print(f"\nTesting: {endpoint}")
    try:
        response = requests.get(endpoint, headers=headers)
        print(f"  Status: {response.status_code}")
        print(f"  Content-Type: {response.headers.get('content-type', 'Not set')}")
        if response.status_code == 200:
            print(f"  Success! Response length: {len(response.text)}")
            if response.text:
                print(f"  First 200 chars: {response.text[:200]}")
                try:
                    data = response.json()
                    print(f"  JSON keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
                except:
                    print("  Not valid JSON")
        else:
            print(f"  Error: {response.text[:100] if response.text else 'Empty response'}")
    except Exception as e:
        print(f"  Exception: {e}")



Testing: https://www.geoguessr.com/api/v3/challenges/FbxiQzxzq9XuwwY2
  Status: 200
  Content-Type: application/json; charset=utf-8
  Success! Response length: 2580
  First 200 chars: {"challenge":{"token":"FbxiQzxzq9XuwwY2","mapSlug":"world","roundCount":5,"timeLimit":0,"forbidMoving":false,"forbidZooming":false,"forbidRotating":false,"numberOfParticipants":31,"gameMode":"standard
  JSON keys: ['challenge', 'map', 'creator']

Testing: https://www.geoguessr.com/api/v3/challenges/FbxiQzxzq9XuwwY2/leaderboard
  Status: 404
  Content-Type: Not set
  Error: Empty response

Testing: https://www.geoguessr.com/api/v3/results/FbxiQzxzq9XuwwY2
  Status: 404
  Content-Type: Not set
  Error: Empty response

Testing: https://www.geoguessr.com/api/v3/results/highscores/FbxiQzxzq9XuwwY2
  Status: 401
  Content-Type: Not set
  Error: Empty response

Testing: https://www.geoguessr.com/api/v3/results/scores/FbxiQzxzq9XuwwY2/0/26
  Status: 404
  Content-Type: Not set
  Error: Empty response


In [13]:
# Let's also test if the challenge ID is valid by checking the challenge page
challenge_id = "FbxiQzxzq9XuwwY2"

# Test if we can access the challenge page
challenge_url = f"https://www.geoguessr.com/challenge/{challenge_id}"
print(f"Testing challenge page: {challenge_url}")

try:
    response = requests.get(challenge_url, headers=headers)
    print(f"Challenge page status: {response.status_code}")
    if response.status_code == 200:
        print("Challenge page is accessible")
        # Look for any API endpoints in the page source
        if "api/v3" in response.text:
            print("Found API references in page source")
        else:
            print("No API references found in page source")
    else:
        print(f"Challenge page error: {response.text[:200] if response.text else 'Empty response'}")
except Exception as e:
    print(f"Exception accessing challenge page: {e}")

# Let's also try to get the challenge metadata first
print(f"\nTrying to get challenge metadata...")
try:
    meta_response = requests.get(f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}", headers=headers)
    print(f"Metadata status: {meta_response.status_code}")
    if meta_response.status_code == 200:
        meta_data = meta_response.json()
        print(f"Challenge metadata: {meta_data}")
    else:
        print(f"Metadata error: {meta_response.text[:200] if meta_response.text else 'Empty response'}")
except Exception as e:
    print(f"Exception getting metadata: {e}")


Testing challenge page: https://www.geoguessr.com/challenge/FbxiQzxzq9XuwwY2
Challenge page status: 200
Challenge page is accessible
No API references found in page source

Trying to get challenge metadata...
Metadata status: 200
Challenge metadata: {'challenge': {'token': 'FbxiQzxzq9XuwwY2', 'mapSlug': 'world', 'roundCount': 5, 'timeLimit': 0, 'forbidMoving': False, 'forbidZooming': False, 'forbidRotating': False, 'numberOfParticipants': 31, 'gameMode': 'standard', 'challengeType': 0, 'streakType': 'countrystreak', 'accessLevel': 0}, 'map': {'id': '52d15ff6850ffb3b847cb3b4', 'name': 'World', 'slug': 'world', 'description': 'Embark on a journey that takes you all over the world. From the most desolate roads in Australia to  the busy, bustling streets of New York City.', 'url': '/maps/world', 'playUrl': '/world/play', 'published': True, 'banned': False, 'images': {'backgroundLarge': 'map/3f950f0318b9086b1b9ec591206dfdd8.jpg', 'incomplete': False}, 'bounds': {'min': {'lat': -54.853378, '

In [14]:
# Working solution using the approach from your hello.py file
def get_challenge_results(challenge_id, ncfa_cookie):
    """
    Get challenge results using the working approach from hello.py
    """
    import requests
    
    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "Referer": f"https://www.geoguessr.com/challenge/{challenge_id}",
    })
    # Set the cookie properly
    s.cookies.set("ncfa", ncfa_cookie, domain=".geoguessr.com")

    # 1) Get challenge metadata (we know this works)
    meta = s.get(f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}").json()
    print(f"Challenge: {meta['challenge']['token']}")
    print(f"Map: {meta['map']['name']}")
    print(f"Participants: {meta['challenge']['numberOfParticipants']}")

    # 2) Try different results endpoints
    candidates = [
        f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/leaderboard",
        f"https://www.geoguessr.com/api/v3/results/{challenge_id}",
        f"https://www.geoguessr.com/api/v3/results/highscores/{challenge_id}",
        f"https://www.geoguessr.com/api/v3/results/scores/{challenge_id}/0/26"
    ]
    
    results = None
    working_endpoint = None
    
    for url in candidates:
        print(f"\nTrying: {url}")
        r = s.get(url)
        print(f"  Status: {r.status_code}")
        if r.ok and r.headers.get("content-type","").startswith("application/json"):
            try:
                results = r.json()
                working_endpoint = url
                print(f"  Success! Got {len(results) if isinstance(results, list) else 'data'}")
                break
            except:
                print(f"  JSON decode error")
        else:
            print(f"  Failed: {r.text[:100] if r.text else 'Empty response'}")
    
    if results is None:
        print("No working results endpoint found")
        return meta, []
    
    print(f"\nWorking endpoint: {working_endpoint}")
    
    # Parse results
    entries = []
    if isinstance(results, list):
        data_list = results
    else:
        data_list = results.get("items", [])
    
    for row in data_list:
        entry = {
            "username": row.get("player", {}).get("nick") if isinstance(row.get("player"), dict) else row.get("playerNick") or row.get("nick"),
            "userId": row.get("player", {}).get("id") if isinstance(row.get("player"), dict) else row.get("playerId"),
            "total": row.get("totalScore") or row.get("score") or row.get("total"),
            "timeMs": row.get("time") or row.get("totalTime"),
            "gameId": row.get("gameId") or row.get("id"),
        }
        entries.append(entry)
    
    return meta, entries

# Test the working solution
challenge_id = "FbxiQzxzq9XuwwY2"
cookie = os.getenv("GEOGUESSR_COOKIE")
if cookie.startswith("_ncfa="):
    cookie = cookie[6:]

meta, results = get_challenge_results(challenge_id, cookie)
print(f"\nFinal results: {len(results)} entries")
for i, entry in enumerate(results):
    print(f"{i+1}. {entry}")


Challenge: FbxiQzxzq9XuwwY2
Map: World
Participants: 31

Trying: https://www.geoguessr.com/api/v3/challenges/FbxiQzxzq9XuwwY2/leaderboard
  Status: 404
  Failed: Empty response

Trying: https://www.geoguessr.com/api/v3/results/FbxiQzxzq9XuwwY2
  Status: 404
  Failed: Empty response

Trying: https://www.geoguessr.com/api/v3/results/highscores/FbxiQzxzq9XuwwY2
  Status: 401
  Failed: Empty response

Trying: https://www.geoguessr.com/api/v3/results/scores/FbxiQzxzq9XuwwY2/0/26
  Status: 404
  Failed: Empty response
No working results endpoint found

Final results: 0 entries


In [ ]:
# Let's try a more comprehensive approach to find the correct API endpoint
challenge_id = "FbxiQzxzq9XuwwY2"

# First, let's check what the challenge page actually loads
import requests
from bs4 import BeautifulSoup

challenge_url = f"https://www.geoguessr.com/challenge/{challenge_id}"
print(f"Analyzing challenge page: {challenge_url}")

# Get the challenge page with proper headers
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.99 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

# Set up session with cookie
s = requests.Session()
s.headers.update(headers)
cookie = os.getenv("GEOGUESSR_COOKIE")
if cookie.startswith("_ncfa="):
    cookie = cookie[6:]
s.cookies.set("ncfa", cookie, domain=".geoguessr.com")

# Get the challenge page
response = s.get(challenge_url)
print(f"Challenge page status: {response.status_code}")

if response.status_code == 200:
    # Look for API calls in the page source
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Look for script tags that might contain API calls
    scripts = soup.find_all('script')
    api_calls = []
    
    for script in scripts:
        if script.string and 'api/v3' in script.string:
            lines = script.string.split('\n')
            for line in lines:
                if 'api/v3' in line and ('results' in line or 'leaderboard' in line or 'scores' in line):
                    api_calls.append(line.strip())
    
    print(f"Found {len(api_calls)} potential API calls in page source:")
    for call in api_calls[:10]:  # Show first 10
        print(f"  {call}")
    
    # Also look for any data attributes or other clues
    if 'leaderboard' in response.text.lower():
        print("Found 'leaderboard' in page source")
    if 'results' in response.text.lower():
        print("Found 'results' in page source")
    if 'scores' in response.text.lower():
        print("Found 'scores' in page source")


In [ ]:
# Let's also try some alternative API patterns that might work
challenge_id = "FbxiQzxzq9XuwwY2"

# Test different API patterns
alternative_endpoints = [
    # Different result patterns
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/results",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/participants",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/games",
    
    # Different score patterns
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/scores",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/scores/0/50",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/scores/0/100",
    
    # Different leaderboard patterns
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/leaderboard/0/50",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/leaderboard/0/100",
    
    # Results with different parameters
    f"https://www.geoguessr.com/api/v3/results/{challenge_id}/0/50",
    f"https://www.geoguessr.com/api/v3/results/{challenge_id}/0/100",
]

print("Testing alternative API endpoints...")
for endpoint in alternative_endpoints:
    print(f"\nTesting: {endpoint}")
    try:
        r = s.get(endpoint)
        print(f"  Status: {r.status_code}")
        if r.status_code == 200:
            print(f"  Success! Content-Type: {r.headers.get('content-type', 'Not set')}")
            if r.text:
                try:
                    data = r.json()
                    print(f"  JSON keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
                    if isinstance(data, list):
                        print(f"  List length: {len(data)}")
                    elif isinstance(data, dict) and 'items' in data:
                        print(f"  Items length: {len(data['items'])}")
                except:
                    print("  Not valid JSON")
        else:
            print(f"  Error: {r.text[:100] if r.text else 'Empty response'}")
    except Exception as e:
        print(f"  Exception: {e}")


In [15]:
# Based on the GeoGuessr-API repository, let's test the correct v3/v4 endpoints
challenge_id = "FbxiQzxzq9XuwwY2"

# From the repository, let's test these relevant endpoints:
correct_endpoints = [
    # v3 endpoints that might work for challenges
    f"https://www.geoguessr.com/api/v3/games",
    f"https://www.geoguessr.com/api/v3/scores/maps",
    
    # v4 endpoints that might be more current
    f"https://www.geoguessr.com/api/v4/games/infinity/challenges",
    f"https://www.geoguessr.com/api/v4/games/infinity/history",
    
    # Let's also try some variations that might work
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/games",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/scores",
    f"https://www.geoguessr.com/api/v3/challenges/{challenge_id}/participants",
]

print("Testing endpoints from the GeoGuessr-API repository...")
for endpoint in correct_endpoints:
    print(f"\nTesting: {endpoint}")
    try:
        r = s.get(endpoint)
        print(f"  Status: {r.status_code}")
        if r.status_code == 200:
            print(f"  Success! Content-Type: {r.headers.get('content-type', 'Not set')}")
            if r.text:
                try:
                    data = r.json()
                    print(f"  JSON keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
                    if isinstance(data, list):
                        print(f"  List length: {len(data)}")
                        if len(data) > 0:
                            print(f"  First item keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'Not a dict'}")
                    elif isinstance(data, dict) and 'items' in data:
                        print(f"  Items length: {len(data['items'])}")
                        if len(data['items']) > 0:
                            print(f"  First item keys: {list(data['items'][0].keys()) if isinstance(data['items'][0], dict) else 'Not a dict'}")
                except Exception as e:
                    print(f"  JSON decode error: {e}")
        else:
            print(f"  Error: {r.text[:100] if r.text else 'Empty response'}")
    except Exception as e:
        print(f"  Exception: {e}")


Testing endpoints from the GeoGuessr-API repository...

Testing: https://www.geoguessr.com/api/v3/games
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/scores/maps
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v4/games/infinity/challenges
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v4/games/infinity/history
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/challenges/FbxiQzxzq9XuwwY2/games
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/challenges/FbxiQzxzq9XuwwY2/scores
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/challenges/FbxiQzxzq9XuwwY2/participants
  Exception: name 's' is not defined


In [16]:
# Let's also try to get games data with the challenge ID as a parameter
challenge_id = "FbxiQzxzq9XuwwY2"

# Test different ways to query games with challenge ID
games_endpoints = [
    f"https://www.geoguessr.com/api/v3/games?challenge={challenge_id}",
    f"https://www.geoguessr.com/api/v3/games?challengeId={challenge_id}",
    f"https://www.geoguessr.com/api/v3/games?challengeToken={challenge_id}",
    f"https://www.geoguessr.com/api/v3/games?token={challenge_id}",
    f"https://www.geoguessr.com/api/v3/games?challenge_id={challenge_id}",
]

print("Testing games endpoints with challenge ID parameters...")
for endpoint in games_endpoints:
    print(f"\nTesting: {endpoint}")
    try:
        r = s.get(endpoint)
        print(f"  Status: {r.status_code}")
        if r.status_code == 200:
            print(f"  Success! Content-Type: {r.headers.get('content-type', 'Not set')}")
            if r.text:
                try:
                    data = r.json()
                    print(f"  JSON keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
                    if isinstance(data, list):
                        print(f"  List length: {len(data)}")
                        if len(data) > 0:
                            print(f"  First item keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'Not a dict'}")
                    elif isinstance(data, dict) and 'items' in data:
                        print(f"  Items length: {len(data['items'])}")
                        if len(data['items']) > 0:
                            print(f"  First item keys: {list(data['items'][0].keys()) if isinstance(data['items'][0], dict) else 'Not a dict'}")
                except Exception as e:
                    print(f"  JSON decode error: {e}")
        else:
            print(f"  Error: {r.text[:100] if r.text else 'Empty response'}")
    except Exception as e:
        print(f"  Exception: {e}")

# Also try to get all games and filter by challenge ID
print(f"\nTrying to get all games and filter by challenge ID...")
try:
    r = s.get("https://www.geoguessr.com/api/v3/games")
    print(f"  Status: {r.status_code}")
    if r.status_code == 200:
        data = r.json()
        print(f"  Got {len(data) if isinstance(data, list) else 'data'}")
        if isinstance(data, list):
            # Look for games with our challenge ID
            challenge_games = [game for game in data if str(game.get('challenge', {}).get('token', '')) == challenge_id]
            print(f"  Found {len(challenge_games)} games for challenge {challenge_id}")
            if challenge_games:
                print(f"  First game keys: {list(challenge_games[0].keys())}")
        elif isinstance(data, dict) and 'items' in data:
            challenge_games = [game for game in data['items'] if str(game.get('challenge', {}).get('token', '')) == challenge_id]
            print(f"  Found {len(challenge_games)} games for challenge {challenge_id}")
            if challenge_games:
                print(f"  First game keys: {list(challenge_games[0].keys())}")
except Exception as e:
    print(f"  Exception: {e}")


Testing games endpoints with challenge ID parameters...

Testing: https://www.geoguessr.com/api/v3/games?challenge=FbxiQzxzq9XuwwY2
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/games?challengeId=FbxiQzxzq9XuwwY2
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/games?challengeToken=FbxiQzxzq9XuwwY2
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/games?token=FbxiQzxzq9XuwwY2
  Exception: name 's' is not defined

Testing: https://www.geoguessr.com/api/v3/games?challenge_id=FbxiQzxzq9XuwwY2
  Exception: name 's' is not defined

Trying to get all games and filter by challenge ID...
  Exception: name 's' is not defined
